In [1]:
%load_ext autoreload
%autoreload 2

In [88]:
import argparse
import functools
import hashlib
import itertools
import json
import os
import sys
from copy import deepcopy
from functools import partial
from glob import glob
from multiprocessing import Pool
from time import sleep

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import portalocker as pl
import seaborn as sns
from matplotlib import cm
from tqdm import tqdm
from matplotlib.lines import Line2D

from plot_metrics import preprocess_metrics
from util.metrics_utils import *

sns.set_style('whitegrid')

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [3]:
img_path = '/home/giampaolo/facil-tc/hf-project/final_results/results/images/decay/'
img_path = '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/final_results/results/images/decay/'
img_path = '/home/alfredo/facil-tc/src/'

if not os.path.exists(img_path):
    os.makedirs(img_path)

In [4]:
# def barplot_decay(data, base, incr, metric, approach, savefig):
#     import matplotlib as mpl
#     # https://stackoverflow.com/questions/6615002/given-an-rgb-value-how-do-i-create-a-tint-or-shade/6615053
#     # For tints, calculate (255 - previous value), 
#     # multiply that by 1/4, 1/2, 3/4, etc. (the greater the factor, the lighter the tint),
#     # and add that to the previous value (assuming each.component is a 8-bit integer).
#     def make_lighter(rgba_color, weight=0.5):
#         delta = 255 - 255 * rgba_color
#         delta = delta * weight
#         return (255 * rgba_color + delta) / 255

#     colors_top = 'lightgray'
#     colors_bottom = 'gray'
#     xticks = [base + (i + 1) * incr for i in np.arange(len(data[data['Type'] == 'New']))]
#     fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(8, 5))

#     font_size = 14
#     mpl.rcParams.update({
#         'font.size': font_size,
#         'xtick.labelsize': font_size,
#         'ytick.labelsize': font_size,
#     })

#     col_idx = 0
#     #############################
#     ### TOP histo
#     #############################
#     ax = axes[0]
#     data[data['Type'] == 'Old'].plot(ax=ax, kind='bar', rot=0, color=colors_top)
#     #, width=1)#, yerr=df_lst['std_old'].values)
#     ax.xaxis.set_ticklabels(xticks)

#     ax.set_ylim(0,100)
#     ax.set_title('%s Decay: %s base: %s, incr: %s' % (metric, appr_dict[approach], base, incr))
#     ax.spines['top'].set_visible(False)
#     ax.spines['left'].set_visible(False)
#     ax.spines['right'].set_visible(False)

#     if col_idx == 0:
#         ax.set_ylabel('avg f1 drop : old')
#     else:
#         ax.yaxis.set_ticklabels([])   
#     for p in ax.patches:
#             ax.annotate(
#                 f'{p.get_height():.2f}', 
#                (p.get_x() + p.get_width() / 2., p.get_height()-.025), 
#                ha = 'center', va = 'center', 
#                xytext = (0, 9), 
#                textcoords = 'offset points'
#             )

#     #############################
#     ### BOTTOM histo
#     #############################
#     ax = axes[1]
#     data[data['Type'] == 'New'].plot(ax=ax, kind='bar', color=colors_bottom)
#     #, width=1)#, yerr=df_lst['std_new'].values)

#     ax.set_ylim(0, 100)
#     ax.invert_yaxis()
#     ax.xaxis.set_ticks_position('top')
#     ax.xaxis.set_ticklabels([])

#     ax.spines['bottom'].set_visible(False)
#     ax.spines['left'].set_visible(False)
#     ax.spines['right'].set_visible(False)

#     if col_idx == 0:
#         ax.set_ylabel('avg f1 drop : new')
#     else:
#         ax.yaxis.set_ticklabels([])

#     for p in ax.patches:
#         ax.annotate(
#             f'{p.get_height():.2f}', 
#                (p.get_x() + p.get_width() / 2., p.get_height()+ 15 ), 
#                 ha = 'center', va = 'center', 
#                 xytext = (0, 9), 
#                 textcoords = 'offset points'
#             )

#     if savefig:
#         plt.savefig('%s.pdf' % savefig, dpi=300, bbox_inches='tight')
#         plt.savefig('%s.pdf' % savefig, bbox_inches='tight')
    

In [5]:
# base_incr = [(20,5), (34,1), (34,2), (34,3)]
# metric = 'F1 Score Drop'
# ord_list = ['lucir', 'icarlp', 'ssil', 'bic', 'icarl', 'ewc', 'eeil', 'il2m', 'wu2022']
# for approach in ord_list:
#     print(approach)
#     for b,i in base_incr:
#         exp_name = 'mirage_generic_%s_*_base%s_incr%s_*' % (approach,b,i)
#         results_path = '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/final_results/results'
#         analysis = 'per_class_metrics'
#         df_filenames = glob('%s/*%s*/**/*%s.parquet' % (results_path, exp_name, analysis), recursive=True)
#         df_filenames = remove_duplicates (df_filenames)
#         df_filenames_scratch = glob('%s_UB/**/*%s.parquet' % (results_path, analysis), recursive=True)
#         df = load_data(df_filenames, 'f1_score')
#         df_scratch = load_scratch_data(df_filenames_scratch, 'f1_score', b, i)
#         df = compute_drops(df, df_scratch)
#         barplot_decay(df, b, i, metric, approach, savefig=os.path.join(
#             img_path, '%s_decay_%s_base_%s_incr_%s' % (metric.lower().replace(' ', '-'), approach, b, i)))

In [84]:
from util.colour_mapper import colour_mapper
import matplotlib as mpl

def lineplot_decay(data, data_forg, base,incr,metric, t, max_old, max_new, ord_list, savefig=None):
    # https://stackoverflow.com/questions/6615002/given-an-rgb-value-how-do-i-create-a-tint-or-shade/6615053
    # For tints, calculate (255 - previous value), 
    # multiply that by 1/4, 1/2, 3/4, etc. (the greater the factor, the lighter the tint),
    # and add that to the previous value (assuming each.component is a 8-bit integer).
    def make_lighter(rgba_color, weight=0.5):
        delta = 255 - 255*rgba_color
        delta = delta*weight
        return (255*rgba_color + delta)/255
    
    plt.rcParams['xtick.bottom'] = True
    plt.rcParams['ytick.left'] = True

    # colors_top = plt.get_cmap('plasma', 3).colors
    # colors_bottom = [[make_lighter(val, 0.6) for val in tpl] for tpl in colors_top]
    palette = dict([(appr_dict[k], colour_mapper()[k]) for k in ord_list])
    colors_top = 'lightgray'
    colors_bottom = 'gray'
    xticks = [int(base+(i+1)*incr) for i in np.arange((40-base)/incr)]
    fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(7,7))

    font_size = 14
    mpl.rcParams.update({
        'font.size': font_size,
        'xtick.labelsize': font_size,
        'ytick.labelsize': font_size,
    })

    col_idx = 0
    #############################
    ### TOP histo
    #############################
    ax = axes[0]
    ax.set_xticks(range(int(xticks[0]), int(xticks[-1])+incr, incr))
    ax.xaxis.set_ticklabels(xticks)
    ax.xaxis.set_ticklabels([])
    ax.tick_params(axis='x', which='major', length=5)
    sns.lineplot(x='#Apps',y='F1 Score Drop', data=data[data['Type']=='New'], ax=ax, hue='Approach',
                 markers=True, style='Approach', linewidth = 3, markersize=15, ci=None, palette=palette,
                mec='none')

    leg = ax.legend(title='Approach', bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    leg.get_frame().set_linewidth(0.)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set(xlabel=None)
    
    if col_idx == 0:
        ax.set_ylabel('Intransigence [%]')
    else:
        ax.yaxis.set_ticklabels([])   
    for p in ax.patches:
            ax.annotate(
                f'{p.get_height():.2f}', 
               (p.get_x() + p.get_width() / 2., p.get_height()-.025), 
               ha = 'center', va = 'center', 
               xytext = (0, 9), 
               textcoords = 'offset points'
            )

    #############################
    ### BOTTOM histo
    #############################
    ax = axes[1]
    sns.lineplot(x='#Apps', y='F1 Score Drop', data=data[data['Type']=='Old'], ax=ax, hue='Approach',
                 markers=True, style='Approach', linewidth = 3, markersize=15, ci=None, palette=palette,
                mec='none')

    ax.xaxis.set_ticks_position('top')
    ax.set_xticks(range(int(xticks[0]), int(xticks[-1])+incr, incr))
    ax.xaxis.set_ticklabels([])
    ax.tick_params(axis='x', which='major', length=5)

    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.legend_ = None
    ax.set(xlabel=None)

    if col_idx == 0:
        ax.set_ylabel('Drop Old [%]')
    else:
        ax.yaxis.set_ticklabels([])

    for p in ax.patches:
        ax.annotate(
            f'{p.get_height():.2f}', 
               (p.get_x() + p.get_width() / 2., p.get_height()+ 15 ), 
                ha = 'center', va = 'center', 
                xytext = (0, 9), 
                textcoords = 'offset points'
            )

    #############################
    ### BOTTOM histo
    #############################
    ax = axes[2]
    sns.lineplot(x='#Apps', y='F1 Score Forgetting', data=data_forg[data_forg['Type']=='Old'], ax=ax, hue='Approach',
                 markers=True, style='Approach', linewidth = 3, markersize=15, palette=palette,
                mec='none',ci=None)
    
    ax.set_xticks(xticks)
    ax.xaxis.set_ticklabels(xticks)
    ax.tick_params(axis='x', which='major', length=5)
    ax.spines['bottom'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.legend_ = None

    if col_idx == 0:
        ax.set_ylabel('Forgetting [%]')
        ax.yaxis.set_label_coords(-0.08,0.45)
    else:
        ax.yaxis.set_ticklabels([])

    for p in ax.patches:
        ax.annotate(
            f'{p.get_height():.2f}', 
               (p.get_x() + p.get_width() / 2. -10, p.get_height()+ 15 ), 
                ha = 'center', va = 'center', 
                xytext = (10, 9),
                textcoords = 'offset points'
            )

    
    if savefig:
        plt.savefig('%s.pdf' % savefig, bbox_inches='tight')

In [85]:
from util.colour_mapper import colour_mapper
import matplotlib as mpl

def lineplot_decay_per_approach(data, data_forg, base,incr,metric, t, max_old, max_new, ord_list, savefig=None):
    # https://stackoverflow.com/questions/6615002/given-an-rgb-value-how-do-i-create-a-tint-or-shade/6615053
    # For tints, calculate (255 - previous value), 
    # multiply that by 1/4, 1/2, 3/4, etc. (the greater the factor, the lighter the tint),
    # and add that to the previous value (assuming each.component is a 8-bit integer).
    
    mark = {1:'o', 2:'s', 3:'p', 5:'^', 10:'>', 20:'<'}

    c = ['navy', 'mediumvioletred', 'rebeccapurple', 'darkolivegreen', 'lime', 'forestgreen']
    p = {1:c[0], 2:c[1], 3:c[2], 5:c[3], 10:c[4], 20:c[5]}
    
    def make_lighter(rgba_color, weight=0.5):
        delta = 255 - 255*rgba_color
        delta = delta*weight
        return (255*rgba_color + delta)/255
    
    plt.rcParams['xtick.bottom'] = True
    plt.rcParams['ytick.left'] = True

    # colors_top = plt.get_cmap('plasma', 3).colors
    # colors_bottom = [[make_lighter(val, 0.6) for val in tpl] for tpl in colors_top]
    
    palette = dict([(appr_dict[k], colour_mapper()[k]) for k in ord_list])
    colors_top = 'lightgray'
    colors_bottom = 'gray'
    xticks = [int(base+(i+1)*incr) for i in np.arange((40-base)/incr)]
    xticks = [25,30,35,36,37,38,39,40]
    fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(7,7))

    font_size = 14
    mpl.rcParams.update({
        'font.size': font_size,
        'xtick.labelsize': font_size,
        'ytick.labelsize': font_size,
    })

    col_idx = 0
    #############################
    ### TOP histo
    #############################
    ax = axes[0]

    sns.lineplot(x='#Apps',y='F1 Score Drop', data=data[data['Type']=='New'], ax=ax, hue='incr',
                 markers=mark, style='incr', linewidth = 1.5, markersize=11, palette=p,
                mec='black',ci=None, alpha=0.75)

    leg = ax.legend(title='Increment', bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    leg.get_frame().set_linewidth(0.)
    ax.set_xticks(xticks)
    ax.xaxis.set_ticklabels(xticks)
    ax.xaxis.set_ticklabels([])

    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set(xlabel=None)

    if col_idx == 0:
        ax.set_ylabel('Intransigence [%]')
    else:
        ax.yaxis.set_ticklabels([])   
    for p in ax.patches:
            ax.annotate(
                f'{p.get_height():.2f}', 
               (p.get_x() + p.get_width() / 2., p.get_height()-.025), 
               ha = 'center', va = 'center', 
               xytext = (0, 9), 
               textcoords = 'offset points'
            )

            
    #############################
    ### MIDDLE histo
    #############################
    ax = axes[1]
    ax.set_xticks(xticks)
    ax.xaxis.set_ticklabels(xticks)
    ax.tick_params(axis='x', which='major', length=5)
    sns.lineplot(x='#Apps',y='F1 Score Drop', data=data[data['Type']=='Old'], ax=ax, hue='incr',
                 markers=mark, style='incr', linewidth = 1.5, markersize=11, palette=p,
                mec='black',ci=None, alpha=0.75)

    ax.xaxis.set_ticklabels([])
    ax.tick_params(axis='x', which='major', length=5)

    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.legend_ = None
    ax.set(xlabel=None)

    if col_idx == 0:
        ax.set_ylabel('Drop Old [%]')
    else:
        ax.yaxis.set_ticklabels([])   
    for p in ax.patches:
            ax.annotate(
                f'{p.get_height():.2f}', 
               (p.get_x() + p.get_width() / 2., p.get_height()-.025), 
               ha = 'center', va = 'center', 
               xytext = (0, 9), 
               textcoords = 'offset points'
            )            
            
    #############################
    ### BOTTOM histo
    #############################
    ax = axes[2]
    sns.lineplot(x='#Apps', y='F1 Score Forgetting', data=data_forg[data_forg['Type']=='Old'], ax=ax, hue='incr',
                 markers=mark, style='incr', linewidth = 1.5, markersize=11, palette=p,
                mec='black',ci=None, alpha=0.75)

    ax.set_xticks(xticks)
    ax.xaxis.set_ticklabels(xticks)
    ax.tick_params(axis='x', which='major', length=5)
    ax.spines['bottom'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.legend_ = None

    if col_idx == 0:
        ax.set_ylabel('Forgetting [%]')
    else:
        ax.yaxis.set_ticklabels([])

    for p in ax.patches:
        ax.annotate(
            f'{p.get_height():.2f}', 
               (p.get_x() + p.get_width() / 2., p.get_height()+ 15 ), 
                ha = 'center', va = 'center', 
                xytext = (0, 9), 
                textcoords = 'offset points'
            )
    
    if savefig:
        plt.savefig('%s.pdf' % savefig, bbox_inches='tight')
        plt.close()

In [90]:
#load data for lineplot
base_incr = [(20,5), (34,1), (34,2), (34,3), (20,10), (20,20)]
metric = 'f1_score'
ord_list = ['lwf','lucir', 'icarlp', 'ssil','il2m','ewc','bic','icarl','eeil','chen2021','backbonefreezing', 'joint']
t='net'
df_lst_cv = pd.DataFrame()
df_forg_all = pd.DataFrame()
for b, i in base_incr:
    for approach in ord_list:
        exp_name = 'mirage_generic_%s_*_base%s_incr%s_*' % (approach,b,i)
        results_path = '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/final_results/results'
        analysis = 'per_class_metrics'
        df_filenames = glob('%s/*%s*/**/*%s.parquet' % (results_path, exp_name, analysis), recursive=True)
        df_filenames_scratch = glob('%s_UB/**/*%s.parquet' % (results_path, analysis), recursive=True)
        df = load_data(df_filenames, metric)
        df_scratch = load_scratch_data(df_filenames_scratch, metric, b, i)
        df = compute_drops(df, df_scratch)
        df['incr'] = [i] * len(df)
        df['base'] = [b] * len(df)
        df_lst_cv=pd.concat([df_lst_cv, df])
        
        analysis = 'per_episode_metrics'
        df_filenames = glob('%s/*%s*/**/*%s.parquet' % (results_path, exp_name, analysis), recursive=True)
        df_forg = load_data(df_filenames, '_'.join(['forgetting',metric]))
        df_forg['incr'] = [i] * len(df_forg)
        df_forg['base'] = [b] * len(df_forg)
        
        df_forg_all = pd.concat([df_forg_all, df_forg])
df_forg_all = df_forg_all.reset_index()        
df_lst_cv = df_lst_cv.reset_index()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 230/230 [00:02<00:00, 99.31it/s]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 230/230 [00:01<00:00, 209.01it/s]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 230/230 [00:01<00:00, 214.79it/s]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 230/230 [00:01<00:00, 228.31it/s]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 230/230 [00:01<00:00, 216.78it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 230/230 [00:01<00:00, 219.24it/s]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 230/230 [00:01<00:00, 211.21it/s]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 230/230 [00:01<00:00, 190.16it/s]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 230/230 [00:01<00:00, 223.20it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 120.31it/s]


In [91]:
#lineplot per approach
l= ['Base', 'Old']
l=['Old']
for t in l:
    base_incr = [(20,5), (34,1), (34,2), (34,3), (20,10), (20,20)]
    metric = 'f1_score'
    ord_list_net = ['lwf','lucir', 'icarlp', 'ssil','il2m','ewc','bic','icarl','eeil','chen2021','backbonefreezing', 'joint']
    max_old = df_lst_cv[df_lst_cv['Type']==t]['%s Drop' % metr_dict[metric]].max()
    max_new = df_lst_cv[df_lst_cv['Type']=='New']['%s Drop' % metr_dict[metric]].max()
    df_forg=pd.DataFrame()
    df=pd.DataFrame()
    total_appr = []
    
    approach_list = ['LwF-GKD', 'LUCIR', 'iCaRL+', 'SSIL', 'IL2M', 'EWC', 'BiC', 'iCaRL', 'EEIL', 'OvA-Ens', 'FT-Mem', 'FZ-Mem']
    for appr in approach_list:
        for (b, i) in base_incr:
            df_lst_temp = df_lst_cv[(df_lst_cv['base'] == b) & (df_lst_cv['incr'] == i) & (df_lst_cv['#Apps'] != b)]
            df_forg_temp = df_forg_all[(df_forg_all['base'] == b) & (df_forg_all['incr'] == i) & (df_forg_all['#Apps'] != b)]

            df = pd.concat([df_lst_temp, df])
            df_forg = pd.concat([df_forg_temp, df_forg])
        df=df[(df['Seed']!=0) & (df['Seed']<11) & (df['Approach']==appr)]
        df_forg=df_forg[(df_forg['Seed']!=0) & (df_forg['Seed']<11) & (df_forg['Approach']==appr)]

        new = df[(df['#Apps'] == 40) & (df['Type']=='New')].groupby('incr').mean()['F1 Score Drop'].reset_index()['F1 Score Drop'].values
        old = df[(df['#Apps'] == 40) & (df['Type']==t)].groupby('incr').mean()['F1 Score Drop'].reset_index()['F1 Score Drop'].values
        forg_old = df_forg[(df_forg['#Apps'] == 40) & (df_forg['Type']=='Old')].groupby('incr').mean()['F1 Score Forgetting'].reset_index()['F1 Score Forgetting'].values
        
        total = list(new)+list(forg_old)
        df_incr_old =  df_forg[(df_forg['Type']=='Old')].groupby(['incr', '#Apps']).mean().reset_index()
        df_incr_new = df[(df['Type']=='New')].groupby(['incr', '#Apps']).mean().reset_index()

        from sklearn.linear_model import LinearRegression
        regressor = LinearRegression()
        coeff_new = []
        coeff_old = []
        for inc in df_incr_old['incr'].unique():
            if inc!=20:
                X_train_old = np.reshape(df_incr_old[df_incr_old['incr']==inc]['#Apps'].values, (-1, 1))
                #Y_train_old = np.reshape(df_incr_old[df_incr_old['incr']==inc]['F1 Score Drop'].values, (-1, 1))
                Y_train_old = np.reshape(df_incr_old[df_incr_old['incr']==inc]['F1 Score Forgetting'].values, (-1, 1))
                regressor.fit(X_train_old, Y_train_old)
                coeff_old.append(regressor.coef_[0][0])
                X_train_new = np.reshape(df_incr_new[df_incr_new['incr']==inc]['#Apps'].values, (-1, 1))
                Y_train_new = np.reshape(df_incr_new[df_incr_new['incr']==inc]['F1 Score Drop'].values, (-1, 1))
                regressor.fit(X_train_new, Y_train_new)
                coeff_new.append(regressor.coef_[0][0])

        coeff_new.append(0)
        coeff_old.append(0)
        total = total + list(coeff_new)
        total = total + list(coeff_old)
        total_appr.append(total)
        lineplot_decay_per_approach(df,df_forg, b, i, metric, t, max_old, max_new, ord_list_net,
                       savefig=os.path.join(img_path, 'LinePDecay_%s_%s' % (appr,t)))

    #from sklearn.cluster import KMeans
    #kmeans = KMeans(n_clusters=4, random_state=0).fit(np.asarray(total_appr))
    #print(t)
    #for i in sorted(range(len(kmeans.labels_)), key=lambda k: kmeans.labels_[k]):
    #    print(approach_list[i], kmeans.labels_[i])#, total_appr[i] )
    #print('\n----------------------------------------------------------\n')

In [92]:
#lineplot comparison
base_incr = [(20,5), (20,10), (20,20), (34,1), (34,2), (34,3)]

base_incr_appr = {(20,5): ['iCaRL', 'OvA-Ens', 'FT-Mem', 'LwF-GKD'],
                  (20,10): ['iCaRL', 'OvA-Ens', 'FT-Mem', 'LwF-GKD'],
                  (20,20): ['iCaRL', 'OvA-Ens', 'FT-Mem', 'LwF-GKD'],
                  (34,1): ['iCaRL', 'OvA-Ens', 'FT-Mem', 'LwF-GKD'],
                  (34,2): ['iCaRL', 'OvA-Ens', 'FT-Mem', 'LwF-GKD'],
                  (34,3): ['iCaRL', 'OvA-Ens', 'FT-Mem', 'LwF-GKD']}
l= ['Base', 'Old']
l= ['Old']
for t in l:
    metric = 'f1_score'
    ord_list_net = ['chen2021', 'icarlp']
    ord_list_net = ['lwfgkd','lucir', 'icarlp', 'ssil','il2m','ewc','bic','icarl','eeil','chen2021','backbonefreezingmem', 'jointmem']

    approach_list = ['LwF-GKD', 'LUCIR', 'iCaRL+', 'SSIL', 'IL2M', 'EWC', 'BiC', 'iCaRL', 'EEIL', 'OvA-Ens', 'FT-Mem', 'FZ-Mem']

    df_lst_cv = df_lst_cv[df_lst_cv['Approach'].isin(approach_list)]
    for (b, i) in base_incr:
        print(b,i)
        total_appr=[]
        df_forg_temp = df_forg_all[(df_forg_all['base'] == b) & (df_forg_all['incr'] == i) & (df_forg_all['#Apps'] != b)]
        df_lst_temp = df_lst_cv[(df_lst_cv['base'] == b) & (df_lst_cv['incr'] == i) & (df_lst_cv['#Apps'] != b)]
        for appr in df_lst_temp['Approach'].unique():
            single_appr=[]
            df=df_lst_temp[(df_lst_temp['Seed']!=0) & (df_lst_temp['Seed']<11) & (df_lst_temp['Approach']==appr)]
            df_forg=df_forg_temp[(df_forg_temp['Seed']!=0) & (df_forg_temp['Seed']<11) & (df_forg_temp['Approach']==appr)]
            new = df[(df['#Apps'] == 40) & (df['Type']=='New')]['F1 Score Drop'].mean()
            old = df[(df['#Apps'] == 40) & (df['Type']==t)]['F1 Score Drop'].mean()
            single_appr.append(new)
            single_appr.append(old)

            if i!=20:
                df_incr_old = df[(df['Type']==t)].groupby(['#Apps']).mean().reset_index()
                df_incr_new = df[(df['Type']=='New')].groupby(['#Apps']).mean().reset_index()
                from sklearn.linear_model import LinearRegression
                regressor = LinearRegression()
                X_train_old = np.reshape(df_incr_old['#Apps'].values, (-1, 1))
                Y_train_old = np.reshape(df_incr_old['F1 Score Drop'].values, (-1, 1))

                regressor.fit(X_train_old, Y_train_old)
                coeff_old = regressor.coef_[0][0]
                X_train_new = np.reshape(df_incr_new['#Apps'].values, (-1, 1))
                Y_train_new = np.reshape(df_incr_new['F1 Score Drop'].values, (-1, 1))
                regressor.fit(X_train_new, Y_train_new)
                coeff_new = regressor.coef_[0][0]
            else:
                coeff_new=0
                coeff_old=0
            single_appr.append(coeff_new)
            single_appr.append(coeff_old)
            total_appr.append(single_appr)

        df_lst_temp=df_lst_temp[df_lst_temp['Seed']!=0]
        lineplot_decay(df_lst_temp[df_lst_temp['Approach'].isin(base_incr_appr[(b,i)])],df_forg_temp[df_forg_temp['Approach'].isin(base_incr_appr[(b,i)])], b, i, metric, t, max_old, max_new, ord_list_net,
                       savefig=os.path.join(img_path, 'LinePDecay_%s_incr%s_%s' % (b, i, t)))

        #from sklearn.cluster import KMeans
        #kmeans = KMeans(n_clusters=3, random_state=0).fit(np.asarray(total_appr))
        #print(t)
        #for i in sorted(range(len(kmeans.labels_)), key=lambda k: kmeans.labels_[k]):
        #    print(df_lst_temp['Approach'].unique()[i], kmeans.labels_[i], total_appr[i] )
        #print('\n----------------------------------------------------------\n')

20 5
20 10
20 20
34 1
34 2
34 3


In [61]:
#lineplot for cv approaches (top 3)
#base_incr = [(20,5), (34,1), (34,2), (34,3)]
#metric = 'f1_score'
#best_3_cv = {(20,5): ['bic', 'lucir', 'ewc'],
#             (34,1): ['bic', 'joint', 'ewc'],
#             (34,2): ['bic', 'joint', 'ewc']}
#t='cv'

#df_lst = pd.DataFrame()

#for b, i in base_incr:
#    for approach in best_3_cv[(b,i)]:
#        exp_name = 'mirage_generic_%s_*_base%s_incr%s_*' % (approach,b,i)
#        results_path = '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/final_results/results'
#        analysis = 'per_class_metrics'
#        df_filenames = glob('%s/*%s*/**/*%s.parquet' % (results_path, exp_name, analysis), recursive=True)
#        df_filenames_scratch = glob('%s_UB/**/*%s.parquet' % (results_path, analysis), recursive=True)
#        df = load_data(df_filenames, metric)
        #analysis = 'per_episode_metrics'
        #df_filenames = glob('%s/*%s*/**/*%s.parquet' % (results_path, exp_name, analysis), recursive=True)
        #df_forg = load_data(df_filenames, '_'.join(['forgetting',metric]))
        #print(df_forg)
#        df_scratch = load_scratch_data(df_filenames_scratch, metric, b, i)
#        df = compute_drops(df, df_scratch)
#        df['incr'] = [i] * len(df)
#        df['base'] = [b] * len(df)
#        df_lst=pd.concat([df_lst, df])
#df_lst = df_lst.reset_index()

In [ ]:
#lineplot for cv approaches (top 3)
#base_incr = [(20,5), (34,1), (34,2)]#, (34,3)]
#metric = 'f1_score'
#best_3_cv = {(20,5): ['bic', 'lucir', 'ewc'],
#             (34,1): ['bic', 'joint', 'ewc'],
#             (34,2): ['bic', 'joint', 'ewc']}
#t='cv'

#max_old = df_lst[df_lst['Type']=='Old']['%s Drop' % metr_dict[metric]].max()
#max_new = df_lst[df_lst['Type']=='New']['%s Drop' % metr_dict[metric]].max()

#for (b, i) in tqdm(base_incr):
#    approaches = [appr_dict[x] for x in best_3_cv[(b, i)]]
#    df_lst_temp = df_lst[(df_lst['base']==b) & (df_lst['incr']==i) & (df_lst['#Apps']!=b) & (df_lst['Approach'].isin(approaches))]
#    df_lst_temp=df_lst_temp[df_lst_temp['Seed']!=0]
#    lineplot_decay(df_lst_temp, b, i, metric, t, max_old, max_new, best_3_cv[(b,i)],
#                  savefig=os.path.join(img_path, 'LinePDecay_base_%s_incr%s_%s' % (b, i, t)))